In [ ]:
## Vineyard WInd Farm
# Sofia Wind Farm AEP Calculation
import time
import numpy as np
import pyproj
import openmdao.api as om



import multiprocessing
import os

n_cpu = multiprocessing.cpu_count()
os.environ['OMP_NUM_THREADS'] = '4'

from openmdao.drivers.scipy_optimizer import ScipyOptimizeDriver


from py_wake.site import UniformWeibullSite
from py_wake.literature.gaussian_models import Bastankhah_PorteAgel_2014
from py_wake.deflection_models import JimenezWakeDeflection
from py_wake.wind_turbines.generic_wind_turbines import GenericWindTurbine
import matplotlib.pyplot as plt
from py_wake.deflection_models import GCLHillDeflection



from topfarm.cost_models.py_wake_wrapper import AEPCostModelComponent

from topfarm import TopFarmProblem
from topfarm.easy_drivers import EasyScipyOptimizeDriver
from topfarm.cost_models.py_wake_wrapper import AEPCostModelComponent


# Define turbine positions
utm_turbines = np.array([
    [41.103121937697836, -70.50715878428475],
    [41.08605135426589, -70.52893886306092],
    [41.06907495813195, -70.55071735654846],
    [41.05216528693592, -70.57223966436968],
    [41.03523311803983, -70.59388571189155],
    [41.01875165952444, -70.57136842618993],
    [41.03553641768292, -70.57197156683259],
    [41.03578916748273, -70.54978939429488],
    [41.05241660207034, -70.55012643633594],
    [41.069444121241446, -70.52846914976273],
    [41.0864685843795, -70.50685386612739],
    [41.05281871576241, -70.5281641735577],
    [41.03608899828052, -70.52788269991984],
    [41.019354943493454, -70.52748060615764],
    [41.019176316971794, -70.5493857747462],
    [41.00298982181863, -70.50494296056287],
    [41.01962872038962, -70.50533901859819],
    [41.03637786046946, -70.50579916640874],
    [41.05309269345054, -70.50611734630984],
    [41.069880463397084, -70.50649926876119],
    [41.103374638445445, -70.48517547048317],
    [41.08668991624634, -70.48481881839271],
    [41.070039408933894, -70.4844961816424],
    [41.053364772701855, -70.48396019118795],
    [41.03668565893784, -70.4836921286794],
    [41.020002317807354, -70.48335705054403],
    [41.05363048869049, -70.46210820708863],
    [41.03685034414846, -70.46163909769962],
    [41.020268168386536, -70.46137103519169],
    [41.00331126908435, -70.48290489738571],
    [40.986636396682826, -70.48262662927168],
    [40.97023183184319, -70.46015498361841],
    [40.986877187953155, -70.46069110863422],
    [41.003576896886955, -70.46095031835783],
    [41.020668891553186, -70.43930427083599],
    [41.037253348297156, -70.43969845409076],
    [41.07030664327593, -70.46234579591253],
    [41.08695092582451, -70.46270633863047],
    [41.053847245421565, -70.43995940596504],
    [41.02075898806328, -70.41723687093905],
    [41.003869314550485, -70.4388159028341],
    [40.987078478692695, -70.43854246248661],
    [41.00410298058662, -70.41684833453292],
    [41.021012293698135, -70.39528758868146],
    [41.037796475679755, -70.39555565118935],
    [41.03749156243427, -70.4174704941248],
    [41.07059241948136, -70.44038953976282],
    [41.10361723844386, -70.46319574320307],
    [41.087174686821584, -70.44070644108437],
    [41.05420266822645, -70.41783741639334],
    [41.05439176304719, -70.39592057025766],
    [41.03806674724768, -70.37360436646567],
    [41.05480537602003, -70.37403329881472],
    [41.07127780700284, -70.37423434569583],
    [41.071176761723024, -70.39634950260667],
    [41.07087387850723, -70.4184638758073],
    [41.12034305324306, -70.46358433572266],
    [41.1371523645893, -70.46391941385804],
    [41.12059547714978, -70.44140216318496],
    [41.10391160045768, -70.44106551680905],
    [41.08755083244358, -70.41881704787065],
    [41.08785389230954, -70.3967018909598]
])




# Transform geographic coordinates to UTM using pyproj
transformer = pyproj.Transformer.from_crs("epsg:4326", "epsg:32619", always_xy=True)
x, y = transformer.transform(utm_turbines[:,1], utm_turbines[:,0])



class Vineyard_southforkwind(UniformWeibullSite):
    def __init__(self, ti=0.07, shear=None):
        
        f = [6.4452, 7.6731, 6.4753, 6.0399, 4.8786, 4.5063, 7.3180, 11.7828, 13.0872, 11.1976, 11.1351, 9.4610]
        a = [9.95, 10.29, 9.58, 8.71, 9.09, 9.40, 10.79, 12.40, 11.52, 10.82, 12.38, 10.35]
        k = [2.178, 1.740, 1.834, 1.748, 1.662, 1.662, 1.846, 2.420, 2.646, 2.178, 2.678, 2.447]



        UniformWeibullSite.__init__(self, np.array(f) / np.sum(f), a, k, ti=ti, shear=shear)
        #self.initial_position = np.array([xinit, yinit]).T
        self.name = "Vineyard Southforkwind"


# site = UniformWeibullSite(p_wd=p_wd, a=weibull_A, k=weibull_k, ti=0.1)

site = Vineyard_southforkwind()

class Haliade13(GenericWindTurbine):
# class ReneTurbine(GenericWindTurbine):

    def __init__(self):
        """
        Parameters
        ----------
        The turbulence intensity Varies around 6-8%
        Hub Height Site Specific
        """
        #power norm in kW
        GenericWindTurbine.__init__(self, name='Haliade-X 13MW', diameter=220, hub_height=135,
                             power_norm=13000, turbulence_intensity=0.08)


wind_turbines = Haliade13()


wf_model = Bastankhah_PorteAgel_2014(site, windTurbines=wind_turbines, k=0.0324555)


#aep = wf_model.aep().sum()
aep = wf_model.aep(x, y).sum()


#print('AEP:', aep)
print('AEP:', aep)






# --- SOFIA WAKE STEERING OPTIMIZATION ---

from topfarm.cost_models.cost_model_wrappers import CostModelComponent
from topfarm.easy_drivers import EasyScipyOptimizeDriver
from topfarm.plotting import NoPlot

# --- Timing the optimization ---
start_time = time.time()

wd = np.arange(0, 360, 1)   
ws = np.arange(4, 24, 1)        




print("\nStarting Wake Steering Optimization (ILK)...")



# Reshape yaw: (n_wt, n_wd, n_ws)
n_wt = len(x)
n_ws = len(ws)
n_wd = len(wd)
#yaw_zero = np.zeros((n_wt, n_wd, n_ws))
yaw_init = np.random.uniform(-5, 5, size=(n_wt, n_wd, n_ws))

# Define AEP function with yaw array shape (n_wt, n_wd, n_ws)
# def aep_func(yaw_ilk):
#     simres = wf_model(x, y, wd=wd, ws=ws, yaw=yaw_ilk, tilt=0) 
#     return simres.aep().sum()

aep_history = []


def aep_func(yaw_ilk):
    simres = wf_model(x, y, wd=wd, ws=ws, yaw=yaw_ilk, tilt=0)
    aep_val = simres.aep().sum()
    aep_history.append(aep_val)
    return aep_val



# Wrap cost component
cost_comp = CostModelComponent(
    input_keys=[('yaw_ilk', yaw_init)],
    n_wt=n_wt, 
    cost_function=aep_func,
    maximize=True,
    objective=True,
    output_keys=[('AEP', 0)]
)


from openmdao.drivers.scipy_optimizer import ScipyOptimizeDriver

driver = ScipyOptimizeDriver()

# Create the OpenMDAO driver directly

driver.options['optimizer'] = 'L-BFGS-B'
driver.options['maxiter'] = 300
driver.options['tol'] = 1e-5

# Use the OpenMDAO driver in the TopFarm problem
problem = TopFarmProblem(
    design_vars={'yaw_ilk': (yaw_init, -30, 30)},
    cost_comp=cost_comp,
    driver=driver,          # ← use the custom driver here
    plot_comp=NoPlot(),
    n_wt=n_wt
)




# Solve it
_, state, _ = problem.optimize()


# End timing
end_time = time.time()
elapsed_time = end_time - start_time


# Results
final_yaw_ilk = state['yaw_ilk']
final_aep = aep_func(final_yaw_ilk)

print("\n Wake Steering Optimization Complete")
print(f" Final AEP (GWh): {final_aep:.6f}")
print(f"⏱ Optimization Time: {elapsed_time:.2f} seconds ({elapsed_time / 60:.2f} minutes)")









# --- Wake Map Visualization for multiple wind directions and speeds ---

# Define selected wind directions and wind speeds (by index)
selected_wd_indices = [0, 1, 2, 3]   # e.g., 0°, 30°, 60°, 90°
selected_ws_indices = [0, 3, 6, 10]  # e.g., 4, 10, 16, 24 m/s

for i, (wd_idx, ws_idx) in enumerate(zip(selected_wd_indices, selected_ws_indices), 1):
    simulation_result = wf_model(
        x, y,
        wd=wd[wd_idx],
        ws=ws[ws_idx],
        yaw=state['yaw_ilk'][:, wd_idx, ws_idx],  # use yaw for current direction/speed
        tilt=0
    )

    plt.figure(figsize=(12, 4))
    simulation_result.flow_map().plot_wake_map(cmap='jet')
    plt.xlabel('x [m]')
    plt.ylabel('y [m]')
    plt.title(f"Wake Map at wd={wd[wd_idx]}°, ws={ws[ws_idx]} m/s")
    plt.tight_layout()
    plt.show()










# Select turbine indices (e.g., first 10)
subset_indices = np.arange(10)  # or pick specific ones: [0, 5, 10, 15, ..., 90]

# Subset turbine positions
x_subset = x[subset_indices]
y_subset = y[subset_indices]

# Define selected wind directions and wind speeds (by index)
selected_wd_indices = [0, 1, 2, 3]   # e.g., 0°, 30°, 60°, 90°
selected_ws_indices = [0, 3, 6, 10]  # e.g., 4, 10, 16, 24 m/s

for i, (wd_idx, ws_idx) in enumerate(zip(selected_wd_indices, selected_ws_indices), 1):
    # Subset the yaw angles
    yaw_subset = state['yaw_ilk'][subset_indices, wd_idx, ws_idx]

    # Run simulation for just the subset
    sim_result = wf_model(
        x_subset, y_subset,
        wd=wd[wd_idx],
        ws=ws[ws_idx],
        yaw=yaw_subset,
        tilt=0
    )

    # Plot
    plt.figure(figsize=(12, 4))
    sim_result.flow_map().plot_wake_map()
    plt.xlabel('x [m]')
    plt.ylabel('y [m]')
    plt.title(f"Wake Map (10 turbines) at wd={wd[wd_idx]}°, ws={ws[ws_idx]} m/s")
    plt.tight_layout()
    plt.show()







# Select turbine indices (e.g., first 10)
subset_indices = np.arange(2)  # or pick specific ones: [0, 5, 10, 15, ..., 90]

# Subset turbine positions
x_subset = x[subset_indices]
y_subset = y[subset_indices]

# Define selected wind directions and wind speeds (by index)
selected_wd_indices = [0, 1, 2, 3]   # e.g., 0°, 30°, 60°, 90°
selected_ws_indices = [0, 3, 6, 10]  # e.g., 4, 10, 16, 24 m/s

for i, (wd_idx, ws_idx) in enumerate(zip(selected_wd_indices, selected_ws_indices), 1):
    # Subset the yaw angles
    yaw_subset = state['yaw_ilk'][subset_indices, wd_idx, ws_idx]

    # Run simulation for just the subset
    sim_result = wf_model(
        x_subset, y_subset,
        wd=wd[wd_idx],
        ws=ws[ws_idx],
        yaw=yaw_subset,
        tilt=0
    )

    # Plot
    plt.figure(figsize=(12, 4))
    sim_result.flow_map().plot_wake_map()
    plt.xlabel('x [m]')
    plt.ylabel('y [m]')
    plt.title(f"Wake Map (10 turbines) at wd={wd[wd_idx]}°, ws={ws[ws_idx]} m/s")
    plt.tight_layout()
    plt.show()















# --- Optional: Plot AEP convergence ---
import matplotlib.pyplot as plt
plt.plot(aep_history, marker='o')
plt.title("AEP Progression During Optimization")
plt.xlabel("Iteration")
plt.ylabel("AEP (GWh)")
plt.grid(True)
plt.tight_layout()
plt.show()

















from scipy.interpolate import make_interp_spline

# Ensure your AEP history is a NumPy array
aep_array = np.array(aep_history)
iterations = np.arange(len(aep_array))

# Create a smoothed x-axis with 1000 points, scaled between real iteration values
x_new = np.linspace(iterations.min(), iterations.max(), 1000)

# Create a cubic spline interpolation of the AEP values
spl = make_interp_spline(iterations, aep_array, k=3)
aep_smooth = spl(x_new)

# Plot
plt.figure(figsize=(6, 5))
plt.plot(x_new, aep_smooth, color='blue', label='Smoothed AEP')
plt.scatter(iterations, aep_array, s=10, color='black', alpha=0.5, label='Actual Points')
plt.title("AEP Progression During Optimization")
plt.xlabel("Iteration")
plt.ylabel("AEP (GWh)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
